<a href="https://colab.research.google.com/github/sabithajaleel/ML-Notebooks/blob/main/DubaiRecommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install lat-lon-parser

In [20]:
from bs4 import BeautifulSoup # library to extract data from a website and used for analytics
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
import requests
from lat_lon_parser import parse

touristAttractionsData = pd.DataFrame(columns=["City", "Attractions", "Latitude", "Longitude"])

r = requests.get("https://en.wikipedia.org/wiki/Tourist_attractions_in_Dubai")

soup = BeautifulSoup(r.text.replace('\n', ''), "html.parser")

lis = soup.find("div", {"class": "mw-content-ltr"}).find("ul").find_all("li")

for li in lis:
  link = li.find("a", href=True)

  attraction = link["title"]

  r = requests.get("https://en.wikipedia.org" + link["href"])
  soup = BeautifulSoup(r.text.replace('\n', ''), "html.parser")

  latitudestr = soup.find('span', {"class": "latitude"}).text
  longitudestr = soup.find('span', {"class": "longitude"}).text
  latitude = parse(latitudestr)
  longitude = parse(longitudestr)

  touristAttractionsData = touristAttractionsData.append({
      "City": "Dubai",
      "Attractions": attraction,
      "Latitude": float(latitude),
      "Longitude": float(longitude)
  }, ignore_index=True)

touristAttractionsData.head()

<ipython-input-20-605116487bb2>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  touristAttractionsData = touristAttractionsData.append({
<ipython-input-20-605116487bb2>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  touristAttractionsData = touristAttractionsData.append({
<ipython-input-20-605116487bb2>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  touristAttractionsData = touristAttractionsData.append({
<ipython-input-20-605116487bb2>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  touristAttractionsData = touristAttractionsData.append({
<ipython-input-20-605116487bb2>:28: FutureWarning: The frame.append method is deprecated

,City,Attractions,Latitude,Longitude
0,Dubai,Burj Khalifa,25.197222,55.274167
1,Dubai,The Dubai Fountain,25.195278,55.275000
2,Dubai,Dubai Frame,25.235556,55.300278
3,Dubai,Museum of the Future,25.219119,55.282100
4,Dubai,Dubai Miracle Garden,25.059722,55.244444


In [ ]:
# call Foursquare API for each location and get the nearby venues, within a 1000-meter radius of that location.
